In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link="https://drive.google.com/open?id=1bgxgK8wkdbl2WVvipCOuB4xe3vd7rp4Y"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('500Cities_cleandata_ReadyForEDA(1).csv')

In [144]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

Q1: Intro

In [0]:
df = pd.read_csv('/content/drive/My Drive/OPIM 5512/Assignment 2/500Cities_cleandata_ReadyForEDA(1).csv')

In [0]:
df=df.dropna()#prep

In [148]:
df.shape

(453, 34)

In [149]:
df.head()

,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
0,1,CA,Folsom,624638,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8,"(38.67504943280, -121.147605753)"
1,2,FL,Largo,1239425,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3,"(27.90909077340, -82.7714203383)"
2,3,CA,Berkeley,606000,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7,"(37.87256787650, -122.274907975)"
3,4,CA,Napa,650258,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2,"(38.29804246490, -122.301093331)"
4,5,FL,Sunrise,1269700,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2,"(26.15468783030, -80.2998411020)"


In [0]:
df = df.drop(['Geolocation','StateAbbr','PlaceName', 'PlaceFIPS', 'Unnamed: 0'], axis=1)# X, covariates: Population2010 and all other health variables

In [151]:
df.head()

,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
0,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8
1,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3
2,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7
3,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2
4,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2


In [0]:
seed = 123 #SEED

In [0]:
from sklearn.utils import shuffle
df = shuffle(df)#SHUFFLE

In [0]:
median=df['Population2010'].median()
#easy median code

In [0]:
df.loc[(df['Population2010'] > median ), 'Population2010Binary'] = "1"
df.loc[(df['Population2010'] < median ), 'Population2010Binary'] = "0"
#making the new columns with the binary based on conditions

In [156]:
df.head()

,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Population2010Binary
146,76377,21.9,22.5,15.3,28.4,72.0,4.4,11.4,6.6,74.3,67.2,54.1,7.0,20.8,17.3,20.9,54.6,13.7,35.9,3.4,33.2,78.6,16.8,32.0,83.5,16.4,40.5,3.3,25.7,0
43,269666,17.6,22.7,16.0,33.2,74.7,5.5,9.1,5.6,74.1,75.4,69.0,6.1,40.6,37.7,19.6,58.0,10.5,35.0,2.9,22.9,81.6,13.2,35.5,85.8,11.4,35.6,3.4,16.0,1
280,67843,8.8,23.9,22.2,28.0,76.6,6.9,7.9,4.9,64.1,82.4,75.3,4.8,53.0,42.3,13.2,74.2,7.9,35.5,2.3,16.6,79.1,10.7,24.8,83.9,9.5,29.5,2.2,7.4,0
120,126326,7.3,24.6,20.5,29.3,75.3,6.6,8.1,5.8,70.0,72.7,67.3,5.4,44.8,39.0,16.6,73.1,8.4,35.2,2.5,22.2,78.3,11.1,32.2,85.8,9.9,28.1,2.7,13.1,1
296,244769,18.3,25.1,18.9,32.5,77.9,7.0,8.1,7.1,76.9,78.7,64.3,7.8,34.0,33.9,20.3,60.4,10.8,34.6,3.1,27.8,78.2,13.0,29.4,79.2,13.3,39.8,3.7,16.1,1


Q2: Spot-checking and k-fold cross-validation (100 points)

In [0]:
df['Population2010Binary'] = df['Population2010Binary'].astype('float')

In [0]:
df=df.dropna()#prep

In [159]:
df.shape

(452, 30)

In [0]:
X = df.drop(['Population2010', 'Population2010Binary'], axis=1)
Y = df['Population2010Binary']

In [161]:
df.dtypes

Population2010              int64
ACCESS2_CrudePrev         float64
ARTHRITIS_CrudePrev       float64
BINGE_CrudePrev           float64
BPHIGH_CrudePrev          float64
BPMED_CrudePrev           float64
CANCER_CrudePrev          float64
CASTHMA_CrudePrev         float64
CHD_CrudePrev             float64
CHECKUP_CrudePrev         float64
CHOLSCREEN_CrudePrev      float64
COLON_SCREEN_CrudePrev    float64
COPD_CrudePrev            float64
COREM_CrudePrev           float64
COREW_CrudePrev           float64
CSMOKING_CrudePrev        float64
DENTAL_CrudePrev          float64
DIABETES_CrudePrev        float64
HIGHCHOL_CrudePrev        float64
KIDNEY_CrudePrev          float64
LPA_CrudePrev             float64
MAMMOUSE_CrudePrev        float64
MHLTH_CrudePrev           float64
OBESITY_CrudePrev         float64
PAPTEST_CrudePrev         float64
PHLTH_CrudePrev           float64
SLEEP_CrudePrev           float64
STROKE_CrudePrev          float64
TEETHLOST_CrudePrev       float64
Population2010

In [0]:
array = df.values
X = X.dropna()
Y = Y.dropna()
validation_size = 0.20
seed = 123
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Evaluate Algorithms
# Test options and evaluation metric
num_folds = 20
seed = 123
scoring = 'neg_mean_squared_error'

In [0]:
models = []
models.append(('BOOST', GradientBoostingClassifier()))
models.append(('CCLASS', DecisionTreeClassifier()))
models.append(('FOREST', RandomForestClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('LOG', LogisticRegression(max_iter=1000000)))#max_iter needed
models.append(('KNNCLASS', KNeighborsClassifier()))
models.append(('GAUS', GaussianNB()))
models.append(('ETREES',ExtraTreesClassifier()))
models.append(('BAGS', BaggingClassifier()))

In [168]:
results = []
names = []
for name, model in models:
	kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

BOOST: 0.607018 (0.100162)
CCLASS: 0.554386 (0.116437)
FOREST: 0.573977 (0.104783)
LDA: 0.562427 (0.103168)
LOG: 0.551170 (0.106901)
KNNCLASS: 0.540351 (0.129501)
GAUS: 0.606871 (0.091467)
ETREES: 0.557164 (0.104560)
BAGS: 0.585088 (0.113648)


In [0]:
#BOOST: 0.607018 (0.100162)
#CCLASS: 0.554386 (0.116437)
#FOREST: 0.573977 (0.104783)
#LDA: 0.562427 (0.103168)
#LOG: 0.551170 (0.106901)
#KNNCLASS: 0.540351 (0.129501)
#GAUS: 0.606871 (0.091467)
#ETREES: 0.557164 (0.104560)
#BAGS: 0.585088 (0.113648)

#THE MODEL THAT PERFORMS THE BEST IS GradientBoostingClassifier, in regards to the best accuracy

In [173]:
Model1 = GradientBoostingClassifier()
Model1.fit(X_train, Y_train)
predictions = Model1.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
#0.5824175824175825 was what I got, which is good as it is not greater than the original BOOST so it is not being overfit

0.5604395604395604
[[29 16]
 [24 22]]
              precision    recall  f1-score   support

         0.0       0.55      0.64      0.59        45
         1.0       0.58      0.48      0.52        46

    accuracy                           0.56        91
   macro avg       0.56      0.56      0.56        91
weighted avg       0.56      0.56      0.56        91



In [180]:
#data
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

#Build the pipeline
pipe = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('boost', GradientBoostingClassifier(random_state=seed))])

param_range = [1, 2, 3, 4]

# Set grid search params
grid_params = [{'boost__criterion': ['friedman_mse'],
		'boost__min_samples_leaf': param_range,
		'boost__max_depth': param_range,
		'boost__min_samples_split': param_range[1:]}]

#construct grid search(when I first ran it kfold had to be 20 but now it takes kfold. Don't really know why.)
gs = GridSearchCV(estimator=pipe,
			param_grid=grid_params,
			scoring='accuracy',
			cv=kfold)

gs.fit(X_train, Y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

# Best params
print('\nBest params:\n', gs.best_params_)

Best accuracy: 0.591

Best params:
 {'boost__criterion': 'friedman_mse', 'boost__max_depth': 2, 'boost__min_samples_leaf': 1, 'boost__min_samples_split': 3}


Best accuracy: 0.591

Best params:
 {'boost__criterion': 'friedman_mse', 'boost__max_depth': 2, 'boost__min_samples_leaf': 1, 'boost__min_samples_split': 3}